In [1]:
# Inspired by:
# https://github.com/johnmyleswhite/sudoku.jl
# https://www.youtube.com/watch?v=E8tkpzDne7I
import numpy as np
from copy import deepcopy
from random import randint, random, uniform

class State:
    def __init__(self, sudoku, starting_positions=None):
        self.sudoku = sudoku
        self.h = 0
        if starting_positions == None:
            self.starting_positions = {}
            for i in range(0, 9, 3):
                for j in range(0, 9, 3):
                    self.starting_positions[str(i) + str(j)] = []
                    sub_box = self.sudoku[i:i + 3, j:j + 3]
                    sub_box = sub_box.flatten()
                    for x in sub_box:
                        if x != 0:
                            self.starting_positions[str(i) + str(j)].append(x)
        else:
            self.starting_positions = starting_positions
        
    def show(self):
        for i in range(len(self.sudoku)):
            for j in range(len(self.sudoku[i])):
                if self.sudoku[i][j] != 0:
                    print(self.sudoku[i][j], end='  ')
                else:
                    print('*', end='  ')
            print('')
            
    def populate_boxes(self): # sve mini kocke popunjava validno vrednostima 1-9
        for i in range(0, 9, 3):
            for j in range(0, 9, 3):
                test_arr = self.sudoku[i:i + 3, j:j + 3]
                test_arr = test_arr.flatten()
#                print(test_arr)
                for x in range(0, 3):
                    for y in range(0, 3):
                        if self.sudoku[i + x][j + y] == 0:
                            while True:
                                random_nr = randint(1, 9)
                                if random_nr not in test_arr:
                                    self.sudoku[i + x][j + y] = random_nr
#                                     test_arr = self.sudoku[i:i + 3, j:j + 3]
#                                     test_arr = test_arr.flatten()
                                    test_arr[x * 3 + y] = random_nr
                                    break
            
    def calc_h(self): # +1 za svaki jedinstveni element u svakoj vrsti/koloni (dakle, kraj = 162 )
        # heuristika redova
        for row in self.sudoku:
            self.h += len(np.unique(row))
        # heuristika kolona
        for column in self.sudoku.T:
            self.h += len(np.unique(column))


In [2]:
def gen_neighbor(state, fail_timer=100000):
    new_state = State(deepcopy(state.sudoku), state.starting_positions)
    chs = [0, 3, 6]
    box_i = chs[randint(0, 2)]
    box_j = chs[randint(0, 2)]
    key = str(box_i) + str(box_j)
    indices_1 = None
    indices_2 = None
    f_1, f_2 = 0, 0
    while fail_timer != 0:
        fail_timer -= 1
        f_1 = randint(0, 8)
        f_2 = randint(0, 8)
        if f_1 == f_2 : # ne smeju da budu isti
            continue
        indices_1 = calc_indices(f_1)
        indices_2 = calc_indices(f_2)
        if new_state.sudoku[box_i + indices_1[0]][box_j + indices_1[1]] in new_state.starting_positions[key] or \
            new_state.sudoku[box_i + indices_2[0]][box_j + indices_2[1]] in new_state.starting_positions[key]: 
            continue # ne smeju da menjaju početne pozicije
        else:
            break
#    print(box_i, box_j, f_1, f_2)    
    if fail_timer != 0:
        new_state.sudoku[box_i + indices_1[0]][box_j + indices_1[1]], new_state.sudoku[box_i + indices_2[0]][box_j + indices_2[1]] = \
         new_state.sudoku[box_i + indices_2[0]][box_j + indices_2[1]], new_state.sudoku[box_i + indices_1[0]][box_j + indices_1[1]]
        return new_state
    else:
        return None
    
def calc_indices(offset, max_v=3):
    possible_vals = [i for i in range(0, max_v)]
    for i in range(0, max_v):
        for j in range(0, max_v):
            if (i * max_v +j) == offset: return i, j

In [34]:
from math import exp

# https://cs.stackexchange.com/questions/11126/initial-temperature-in-simulated-annealing-algorithm
def solve_sudoku(starting_board=None, T=0.5, num_iterations=400000, cooldown=lambda t: 0.99999 * t):
    starting_state = None
    if starting_board == None:
        starting_state = State(np.array([[0, 0, 3, 1, 0, 0, 5, 9, 0], \
                                         [0, 0, 0, 0, 7, 6, 0, 0, 0], \
                                         [2, 0, 7, 5, 0, 0, 0, 1, 8], \
                                         [4, 8, 0, 0, 0, 2, 0, 0, 0], \
                                         [0, 0, 0, 0, 0, 0, 0, 4, 9], \
                                         [0, 0, 1, 0, 0, 0, 8, 0, 3], \
                                         [0, 5, 0, 7, 9, 0, 2, 0, 0], \
                                         [9, 0, 6, 0, 0, 4, 0, 0, 0], \
                                         [0, 0, 8, 0, 3, 0, 0, 7, 0]]))
    else:
        starting_state = State(starting_board) # pod uslovom da je validna
        
    print('Starting board..')
    starting_state.show()
    
    starting_state.populate_boxes()
    starting_state.calc_h()
    current_state = starting_state 
        
    for i in range(0, num_iterations):
        try:
            neighbor = gen_neighbor(current_state)
            neighbor.calc_h()
            
            if neighbor.h == 162:
                print('Done!')
                return neighbor
            
            delta_E = float(neighbor.h - current_state.h)
            
            if delta_E > 0:
                current_state = neighbor
            elif (exp(delta_E/T) >= uniform(0, 1)):
#                print('delte_E:', delta_E, 'Prob: ', exp(delta_E/T), 'Temp: ', T)
                current_state = neighbor         
            
            if i % 1000 == 0:
                print('i: ', i, ' score: ', current_state.h, ' temperature: ' , T)
            T = cooldown(T) 
        except Exception as e:
            s = str(e)
            print(e)
            #print('Error! Try again..')
            return None
    
    print('Failed! Try again..')
    return current_state       

In [39]:
college = np.array([[0, 0, 4, 0, 8, 0, 1, 0, 6], \
                    [8, 6, 0, 0, 0, 0, 0, 4, 0], \
                    [0, 0, 0, 0, 0, 0, 3, 0, 0], \
                    [0, 0, 0, 0, 9, 1, 0, 0, 0], \
                    [0, 9, 7, 0, 5, 0, 0, 0, 0], \
                    [3, 0, 0, 4, 0, 0, 0, 0, 0], \
                    [0, 4, 0, 9, 0, 0, 0, 7, 2], \
                    [0, 5, 1, 0, 0, 0, 0, 3, 8], \
                    [0, 0, 0, 2, 0, 0, 4, 0, 0]])

hard = np.array([[0, 0, 0, 9, 0, 0, 3, 5, 0], \
                 [0, 1, 0, 0, 6, 0, 7, 0, 0], \
                 [0, 0, 0, 0, 0, 0, 0, 2, 0], \
                 [8, 0, 0, 0, 3, 1, 4, 6, 0], \
                 [0, 0, 0, 7, 0, 6, 0, 0, 0], \
                 [0, 9, 6, 8, 4, 0, 0, 0, 2], \
                 [0, 3, 0, 0, 0, 0, 0, 0, 0], \
                 [0, 0, 8, 0, 1, 0, 0, 4, 0], \
                 [0, 7, 2, 0, 0, 4, 0, 0, 0]])

x = solve_sudoku(college, T=0.65555, num_iterations=400000)
x.show()

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  


Starting board..
*  *  4  *  8  *  1  *  6  
8  6  *  *  *  *  *  4  *  
*  *  *  *  *  *  3  *  *  
*  *  *  *  9  1  *  *  *  
*  9  7  *  5  *  *  *  *  
3  *  *  4  *  *  *  *  *  
*  4  *  9  *  *  *  7  2  
*  5  1  *  *  *  *  3  8  
*  *  *  2  *  *  4  *  *  
i:  0  score:  115  temperature:  0.65555
i:  1000  score:  151  temperature:  0.6490271360627234
i:  2000  score:  141  temperature:  0.6425691760289542
i:  3000  score:  144  temperature:  0.6361754740908491
i:  4000  score:  143  temperature:  0.6298453908664906
i:  5000  score:  149  temperature:  0.6235782933359556
i:  6000  score:  145  temperature:  0.6173735547779972
i:  7000  score:  140  temperature:  0.6112305547073835
i:  8000  score:  148  temperature:  0.6051486788128486
i:  9000  score:  145  temperature:  0.599127318895649
i:  10000  score:  147  temperature:  0.5931658728087567
i:  11000  score:  141  temperature:  0.5872637443966332
i:  12000  score:  141  temperature:  0.5814203434356141
i:  13000  scor